# Create Dataset for training

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelWithLMHead, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import transformers
import torch
transformers.set_seed(42)

import wandb

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['WANDB_NOTEBOOK_NAME'] = 'just_finetuning_leo-hessianai.ipynb'
wandb.login(key="247b3da94c9b88bd5e990f1d94799ca3ded57d6b")

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: felix-ml (fml-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/felix/.netrc


True

In [2]:
# Specify the path to your JSON file
file_path = 'berufslexikon_cleaned_manual.json'

# Load the dataset
ds = load_dataset('json', data_files=file_path)

# Specify the model checkpoint
model_checkpoint = "LeoLM/leo-hessianai-7b-chat"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
ds["train"][1]

{'content': '3D-DesignerIn \n 3D-DesignerIn Berufsbereiche: Medien, Grafik, Design, Druck, Kunst, Kunsthandwerk Ausbildungsform: Uni/FH/PH Einstiegsgehalt lt. KV: Gehalt: € 2.210,- bis € 2.730,-  Tätigkeitsmerkmale 3D-DesignerInnen entwerfen, erstellen und programmieren komplexe, dreidimensionale digitale Objekte. Da sind z.B. Figuren für Videospiele und Filme, Grafiken, bewegte Bilder oder räumlich wirkende Darstellungen. Sie animieren auch Standbilder aus 3D-Scans. Außerdem programmieren sie interaktive 3D-Echtzeitgrafiken, zum Beispiel ein dreidimensionales Modell einer realen Szene (Virtuelle Realität).Digitale Objekte dienen zur Visualisierung in allen möglichen Bereichen wie Architektur, Raumplanung, Verkehr und Sport oder für die Visualisierung in technischen Bereichen. Die DesignerInnen nutzen 3D-Modeling-Software (z.B. AutoCad) und Simulationssoftware für interaktive 3D-Echtzeitgrafiken. Besondere Bedeutung hat die Visualisierung von Objekten auch für das E-Learning sowie für 

In [4]:
# count the number of words in the dataset
total_words = 0
for example in ds["train"]:
    total_words += len(example["profession"].split())
    total_words += len(example["content"].split())
print(f"Total number of words in the dataset: {total_words}")


Total number of words in the dataset: 3473496


In [5]:
# As this dataset has no validation split, we will create one
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

# Models
* https://huggingface.co/LeoLM/leo-hessianai-13b - did not fit on 24GB VRAM
    - WARNING:accelerate.big_modeling:You shouldn't move a model when it is dispatched on multiple devices.
    - needs FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn==v2.1.1 --no-build-isolation
* https://huggingface.co/LeoLM/leo-hessianai-7b
* https://huggingface.co/LeoLM/leo-hessianai-7b-chat
    - needs FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn --no-build-isolation

In [6]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=model_checkpoint,
#     torch_dtype=torch.float16,
#     trust_remote_code=True  # True for flash-attn2 else False
# )


In [7]:
# We'll create a tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)#, use_fast=False)

# We'll need padding to have same length sequences in a batch
tokenizer.pad_token = tokenizer.eos_token

# Define a tokenization function that first concatenates text and target
def tokenize_function(example):
    # concatenate all texts and tokenize as one sample
    # merged = str(example["profession"]) + " " + str(example["url"]) + " " + str(example["content"])
    # print(type(merged))
    # print(merged)
    merged =  example["content"]
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch

# def tokenize_function(example):
#     # If the fields are lists of strings, join the strings. Otherwise, use the fields as they are.
#     profession = ' '.join(example["profession"]) if isinstance(example["profession"], list) else example["profession"]
#     url = ' '.join(example["url"]) if isinstance(example["url"], list) else example["url"]
#     content = ' '.join(example["content"]) if isinstance(example["content"], list) else example["content"]

#     merged = profession + " " + url + " " + content
#     batch = tokenizer(merged, padding='max_length', truncation=True, max_length=2048)
#     batch["labels"] = batch["input_ids"].copy()
#     return batch

# Apply it on our dataset, and remove the text columns
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["profession", "url", "content"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1122
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 281
    })
})

In [9]:
# Let's check out one prepared example
print(tokenizer.decode(tokenized_datasets["train"][ 1]["input_ids"]))

<|im_start|> EisenbahnbetriebsleiterIn 
 EisenbahnbetriebsleiterIn Berufsbereiche: Büro, Marketing, Finanz, Recht, Sicherheit / Handel, Logistik, Verkehr Ausbildungsform: Kurz-/Spezialausbildung Einstiegsgehalt: Gehalt: nicht bekannt Beruf merken als PDF anzeigen Hinweis Die betriebsinterne Ausbildung zu diesem Beruf kann nur im Rahmen eines aufrechten Dienstverhältnisses absolviert werden. Tätigkeitsmerkmale EisenbahnbetriebsleiterInnen leiten und überw


# Training

## QLORA
https://freedium.cfd/https%3A%2F%2Fmedium.com%2F%40newhardwarefound%2Fqlora-with-llama-2-ca1b4bcf26f0

In [10]:
# Start a new wandb run
model_name = model_checkpoint.split("/")[-1]
run = wandb.init(project=f"{model_name}-ams-finetuned", job_type="train")

In [11]:
import torch

# If CUDA is available, use it
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
    print(f"Using {torch.cuda.device_count()} GPUs.")
else:
    device = torch.device('cpu')
    print("No GPU available, using the CPU instead.")

Using 1 GPUs.


In [12]:
# # Import the necessary library for loading datasets
# from datasets import load_dataset

# # Specify the name of the dataset
# dataset_name = "timdettmers/openassistant-guanaco"

# # Load the dataset from the specified name and select the "train" split
# dataset = load_dataset(dataset_name, split="train")

In [13]:
dataset = ds["train"]

In [14]:
# Code from taprosoft's github: https://github.com/taprosoft/llm_finetuning/blob/efa6df245fee4faf27206d84802d8f58d4b6e77d/inference.py#L20
from transformers import (AutoModelForCausalLM,
    BitsAndBytesConfig,
    LlamaTokenizer)
import torch
import os

#os.environ["HUGGING_FACE_HUB_TOKEN"] = "{{your_huggingface_hub_token}}"

def load_hf_model(
    base_model,
    mode=8,
    gradient_checkpointing=False,
    device_map="auto",
):
    kwargs = {"device_map": device_map}
    if mode == 8:
        kwargs["quantization_config"] = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=0.0,
        )
    elif mode == 4:
        kwargs["quantization_config"] = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
    elif mode == 16:
        kwargs["torch_dtype"] = torch.float16

    model = AutoModelForCausalLM.from_pretrained(base_model, **kwargs)

    # setup tokenizer
    tokenizer = LlamaTokenizer.from_pretrained(base_model)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
    tokenizer.padding_side = "left"  # Allow batched inference
    return model, tokenizer


In [17]:
# Code from taprosoft's github
from dataclasses import dataclass, field
import transformers
import torch
import copy
from typing import Dict, Sequence
from torch.nn.utils.rnn import pad_sequence

IGNORE_INDEX = -100

@dataclass
class DataCollatorForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer
    source_max_len: int
    target_max_len: int
    train_on_source: bool
    predict_with_generate: bool

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        # Extract elements
        sources = [f"{self.tokenizer.bos_token}{example['input']}" for example in instances]
        targets = [f"{example['output']}{self.tokenizer.eos_token}" for example in instances]
        # Tokenize
        tokenized_sources_with_prompt = self.tokenizer(
            sources,
            max_length=self.source_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        tokenized_targets = self.tokenizer(
            targets,
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        # Build the input and labels for causal LM
        input_ids = []
        labels = []
        for tokenized_source, tokenized_target in zip(
            tokenized_sources_with_prompt['input_ids'],
            tokenized_targets['input_ids']
        ):
            if not self.predict_with_generate:
                input_ids.append(torch.tensor(tokenized_source + tokenized_target))
                if not self.train_on_source:
                    labels.append(
                        torch.tensor([IGNORE_INDEX for _ in range(len(tokenized_source))] + copy.deepcopy(tokenized_target))
                    )
                else:
                    labels.append(torch.tensor(copy.deepcopy(tokenized_source + tokenized_target)))
            else:
                input_ids.append(torch.tensor(tokenized_source))
        # Apply padding
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX) if not self.predict_with_generate else None
        data_dict = {
            'input_ids': input_ids,
            'attention_mask':input_ids.ne(self.tokenizer.pad_token_id),
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict

data_collator = DataCollatorForCausalLM(
        tokenizer=tokenizer,
        source_max_len=280,
        target_max_len=512,
        train_on_source=False,
        predict_with_generate=False,
)

In [15]:
import bitsandbytes as bnb
import torch
import peft

# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


def create_peft_model(model, gradient_checkpointing=True, bf16=True):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )
    from peft.tuners.lora import LoraLayer

    # prepare int-4 model for training
    model = prepare_model_for_kbit_training(
        model, use_gradient_checkpointing=gradient_checkpointing
    )
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # get lora target modules
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    model = get_peft_model(model, peft_config)

    # pre-process the model by upcasting the layer norms in float 32 for
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)

    model.print_trainable_parameters()
    return model

In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Defining the model
model_name = model_checkpoint


model, tokenizer = load_hf_model(
    model_name,
    mode=4,
    gradient_checkpointing=False,
    device_map='auto')



# create peft config
model = create_peft_model(
    model, gradient_checkpointing=False, bf16=False
)

# get all linear layer names
modules = find_all_linear_names(model)


from transformers import TrainingArguments
# Define training args
output_dir = "./results"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    bf16=False,  # Use BF16 if available
    learning_rate=2e-4,
    num_train_epochs=1,
    optim="paged_adamw_8bit", #"adamw_torch" if not mode = 4,8
    gradient_checkpointing=False,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    remove_unused_columns=False,
)



# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.84s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Found 7 modules to quantize: ['q_proj', 'down_proj', 'o_proj', 'k_proj', 'gate_proj', 'up_proj', 'v_proj']
trainable params: 159,907,840 || all params: 6,899,372,032 || trainable%: 2.3177158625209793


In [24]:
trainer.train()
#save the model
trainer.save_model(f"{model_name}-ams-finetuned")

KeyboardInterrupt: 

In [ ]:
from peft import PeftModel

# model is the quantized model loaded using load_hf_model in a previous step
model = PeftModel.from_pretrained(model, f"{model_name}-ams-finetuned")

In [22]:
from transformers import GenerationConfig

# sequences = ["<s>[INST] <<SYS>> You are a helpful assistant. <</SYS>>\
# Extract the place names from the given sentence. [\INST]\n\
# The capital of the United States is Washington D.C."]

prompt = "Erkläre mir bitte wie man von Wien nach München kommt"

sequences = [f"<|im_start|>system\
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer. Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>\n\
    <|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"]


inputs = tokenizer(sequences, padding=True, return_tensors="pt").to('cuda')

outputs = model.generate(
    **inputs, 
    generation_config=GenerationConfig(
        do_sample=True,
        max_new_tokens=512,
        top_p=0.99,
        temperature=1e-8,
    )
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

systemDies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer. Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten. 
     user
Erkläre mir bitte wie man von Wien nach München kommt 
 assistant
Natürlich, gerne helfe ich dir dabei, deine Reise von Wien nach München zu planen. Hier sind einige Optionen, die du in Betracht ziehen kannst:

1. Zug: Du kannst den Zug nehmen. Es gibt mehrere Züge von Wien nach München, und du kannst die Verfügbarkeit und die Fahrpläne einfach über die Website der Österreichischen Bundesbahnen (ÖBB) oder der Deutschen Bahn überprüfen. Die Fahrt dauert etwa 4 Stunden.

2. Auto: Wenn du eine längere Reise planst, bietet es sich möglicherweise an, dein eigenes Auto mitzubringen. Von Wien aus kannst du in etwa 4 Stunden nach München fahren. Alternativ kannst du ein Auto von einem Autovermietungsunternehmen mieten.

3. Flugzeug: Du kannst einen Flug von Wien nach München nehmen. Viele Fluggesellschaften 

In [18]:
# from transformers import pipeline
# import torch

# system_prompt = """<|im_start|>system
# Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
# Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>

# """
# prompt_format = "<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
# prompt = "Erkläre mir wie die Fahrradwegesituation in Hamburg ist."

# generator = pipeline(model=model_name, device="cuda", torch_dtype=torch.float16, trust_remote_code=True) # True for flash-attn2 else False
# print(generator(prompt_format.format(prompt=prompt), do_sample=True, top_p=0.95, max_length=8192))


>>>> Flash Attention installed
>>>> Flash RoPE installed


Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.44s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'generated_text': '<|im_start|>user\nErkläre mir wie die Fahrradwegesituation in Hamburg ist.<|im_end|>\n<|im_start|>assistant\nAls KI habe ich keine persönlichen Erfahrungen mit den Fahrradwegesituationen in verschiedenen Städten und Ländern. Ich kann jedoch allgemeine Informationen zu den Fahrradwegen in Hamburg finden und dir zur Verfügung stellen. \n'}]


In [ ]:
# transformers.logging.set_verbosity_error() # suppress tokenizer warnings

# prefix = "Hilf bei der Suche nach einem Beruf:"

# prompts = [
#     "Ich bin 15 Jahre alt, gut im Umgang mit Menschen und möchte gerne einen Beruf erlernen, in dem ich viel mit Menschen zu tun habe.",
#     "Ich bin 30 Jahre alt und mag Tier sehr gerne. Ich möchte gerne einen Beruf erlernen, in dem ich viel mit Tieren zu tun habe.",
#     "Ich bin 19 und möchte etwas technisches machen. Ich bin sehr gut in Mathe und Physik.",
# ]

# table = wandb.Table(columns=["prompt", "generated", "input_tokens", "generated_tokens"])

# for prompt in prompts:
#     # encode the prompt and generate text until the output length (which includes the context length) reaches 50
#     input_ids = tokenizer.encode(prefix + prompt, return_tensors='pt').to(device)
#     output = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
#     generated = tokenizer.decode(output[0], skip_special_tokens=True)
#     table.add_data(prompt, generated, len(input_ids[0]), len(output[0]))
    
# wandb.log({"generated": table})

In [ ]:
wandb.finish()

In [ ]:


# # Importing the required libraries
# import torch

# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# # Defining the name of the Falcon model
# #model_name = "ybelkada/falcon-7b-sharded-bf16"
# model_name = model_checkpoint

# # # Configuring the BitsAndBytes quantization
# # bnb_config = BitsAndBytesConfig(
# # load_in_4bit=True,
# # bnb_4bit_quant_type="nf4",
# # bnb_4bit_compute_dtype=torch.float16,
# # )

# # # Loading the Falcon model with quantization configuration
# # model = AutoModelForCausalLM.from_pretrained(
# # model_name,
# # quantization_config=bnb_config,
# # trust_remote_code=True
# # )

# model, tokenizer = load_hf_model(
#     model_name,
#     mode=4,
#     gradient_checkpointing=False,
#     device_map='auto')

# # Disabling cache usage in the model configuration
# model.config.use_cache = False

# # Load the tokenizer for the Falcon 7B model with remote code trust
# #tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)



# # Set the padding token to be the same as the end-of-sequence token
# tokenizer.pad_token = tokenizer.eos_token

# # Import the necessary module for LoRA configuration
# from peft import LoraConfig

# # Define the parameters for LoRA configuration
# lora_alpha = 16
# lora_dropout = 0.1
# lora_r = 64

# # get all linear layer names
# modules = find_all_linear_names(model)

# # Create the LoRA configuration object
# peft_config = LoraConfig(
# lora_alpha=lora_alpha,
# lora_dropout=lora_dropout,
# r=lora_r,
# bias="none",
# task_type="CAUSAL_LM",
# target_modules=modules,
# # target_modules=[
# #     "q_proj",
# #     "up_proj",
# #     "o_proj",
# #     "k_proj",
# #     "down_proj",
# #     "gate_proj",
# #     "v_proj"
# # ]
# )

# from transformers import TrainingArguments
# # Define the directory to save training results
# output_dir = "./results"

# # Set the batch size per device during training
# per_device_train_batch_size = 4

# # Number of steps to accumulate gradients before updating the model
# gradient_accumulation_steps = 4

# # Choose the optimizer type (e.g., "paged_adamw_32bit")
# optim = "paged_adamw_32bit"

# # Interval to save model checkpoints (every 10 steps)
# save_steps = 10

# # Interval to log training metrics (every 10 steps)
# logging_steps = 10

# # Learning rate for optimization
# learning_rate = 2e-4

# # Maximum gradient norm for gradient clipping
# max_grad_norm = 0.3

# # Maximum number of training steps
# max_steps = 50

# # Warmup ratio for learning rate scheduling
# warmup_ratio = 0.03

# # Type of learning rate scheduler (e.g., "constant")
# lr_scheduler_type = "constant"

# # Create a TrainingArguments object to configure the training process
# training_arguments = TrainingArguments(
# report_to="wandb",
# output_dir=output_dir,
# per_device_train_batch_size=per_device_train_batch_size,
# gradient_accumulation_steps=gradient_accumulation_steps,
# optim=optim,
# save_steps=save_steps,
# logging_steps=logging_steps,
# learning_rate=learning_rate,
# fp16=True,  # Use mixed precision training (16-bit)
# max_grad_norm=max_grad_norm,
# max_steps=max_steps,
# warmup_ratio=warmup_ratio,
# group_by_length=True,
# lr_scheduler_type=lr_scheduler_type,
# )

# # Import the SFTTrainer from the TRL library
# from trl import SFTTrainer

# # Set the maximum sequence length
# max_seq_length = 512

# # Create a trainer instance using SFTTrainer
# trainer = SFTTrainer(
# model=model,
# train_dataset=dataset,
# peft_config=peft_config,
# dataset_text_field="content",
# max_seq_length=max_seq_length,
# tokenizer=tokenizer,
# args=training_arguments,
# )

# # Iterate through the named modules of the trainer's model
# for name, module in trainer.model.named_modules():

# 	# Check if the name contains "norm"
# 	if "norm" in name:
# 		# Convert the module to use torch.float32 data type
# 		module = module.to(torch.float32)
# 		# RuntimeError: FlashAttention only support fp16 and bf16 data type
